In [1]:
import cplex
from inputs.generateinputs import *  
from asignacion_cuadrillas import cargar_instancia, Modelo



In [2]:
generate_inputs()

KeyboardInterrupt: 

In [2]:
# Lectura de datos desde el archivo de entrada
instancia = cargar_instancia('inputs/input_10_30_3_3_0_2.txt')

# Definicion del problema de Cplex
prob = cplex.Cplex()

# Definicion del modelo
model = Modelo(instancia, prob)

# Armamos modelo
model.armar_lp()



In [4]:

# Resolucion del modelo
# model.resolver_lp()

# Obtencion de la solucion
model.mostrar_solucion()

Funcion objetivo:  0.0 (integer optimal solution)
Variables con valor positivo:
